# Optional Workshop RL04: Maths behind the scheme 

This oprional workshop digs deeper into the details of RL mathematics. 

## Model-Based
In a model-based RL, we want to give the agent a model of how the world works (or how it should perceive how the world works). That is, we want to construct mathematical models for state dynamics and rewards. Often we model the states dynamics as a **Markov Process** and model the rewards as the expectations of future rewards.  

### 1. Markov Process
#### Assumption
------------
To construct a Markov process for states dynamics, we assume that the state is Markov. That is, we assume that the state provides just enough information to represent history (i.e. $s_t$ is sufficient statistic of history $h_t$).

So from that we can get:
$$p(s_{t+1}|{h_t,a_t}) = p(s_{t+1}|{s_t,a_t})$$
Recall that $h_t$ contains all information from time 0 to time $t$, whereas $s_t$ contains only the current information. Here we replaced full information with current information to determine future event, (i.e. future is independent of past present). 

Hence we have statisfied the definition of Markov, and thus we can say the states dynamic is a Markov process or Markov chain.

>Why use Markov assumption:
- so agent makes decision based only on current information
    - computational complexity
    - data required
- can always be satisfied by equating $s_t$ and $h_t$, i.e. $s_t = h_t$
- resulting performance

#### Definition 
-----------
Now let's define things more formally, a Markov Process is:

| Markov Process|
|:--------------|
|$S$ is a set of states $s_t \in S$|
|$P$ is dynamics/transition model that specifies $p(s_{t+1} = s'|{s_t=s})$.|

Suppose we have $N$ states $s_1, s_2, ..., s_N$, state dymanics look like: 
        $$ P = 
        \begin{bmatrix}
            p(s_1|s_1) & p(s_2|s_1) & p(s_3|s_1) & \dots  & p(s_N|s_1) \\
            p(s_1|s_2) & p(s_2|s_2) & p(s_3|s_2) & \dots  & p(s_N|s_2) \\
            \vdots & \vdots & \vdots & \ddots & \vdots \\
            p(s_1|s_N) & p(s_2|s_N) & p(s_3|s_N) & \dots  & p(s_N|s_N)
        \end{bmatrix}$$

> Note that the uderscript here doesn't mean time step anymore, but represents different states in the world. So the first row are the probabilities of tranfering to other states from state 1 $s_1$. 

If we simulate the state transitions, we can sample **episodes** like these:
- $s_1,s_2,s_3,s_2,s_3,s_4,s_5.s_5,s_5,...$
- $s_1,s_2,s_3,s_3,s_4,s_5,s_6.s_6,s_7,...$


### 2. Markov Reward Process (MRP)
#### Defintion
-----------
Now plug in the rewards to define a **Markov Reward Process**, i.e.:


| MRP |
|:    |
|$S$ is a set of states $s_t \in S$|
|$P$ is dynamics/transition model that specifies $p(s'|{s})$|
|$R$ is a reward function that specifies current reward $R(s_t = s)=E \lbrack r_t|s_t = s \rbrack$.| 
|$\gamma$ is the discount factor that $\in [0,1]$|

What $R$ looks like depends on the environment. In the frozen lake environment that we use in the exercise, it rewards the agent with 1 if it gets to the safe state, and 0 reward for other states, i.e.: $$R(s_1) = R(s_2) = ... R(s_{N-1}) = 0$$ $$R(s_N) = 1$$
    
>Note again that the uderscript here doesn't mean time step, but represents different states in the world. So the current reward of being in state 1 $s_1$ is 0. 

#### Value Function 
-----------
Now with rewards we can compute the value function: $$V(s_t = s) = E \lbrack r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... |s_t = s \rbrack,$$ which has 3 ways of computing:
- by **simulation**: 
    - simulate a large amount of episodes 
    - compute the value function for each episode
    - take the average of those 
    - pros: requires no states dynamics
- by **analytical way**:
    - using Markov property: 
        $$V(s) = E \lbrack r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... |s_t = s \rbrack$$

        $$ = E \lbrack r_t|s_t = s \rbrack + \gamma E \lbrack r_{t+1} + \gamma r_{t+2} + ... |s_t = s \rbrack $$ 

        $$ = R(s) + \gamma \sum_{s' \in S} p(s'|s)E \lbrack r_{t} + \gamma r_{t+1} + ... |s_{t} = s' \rbrack$$

        $$ = R(s) + \gamma \sum_{s' \in S} p(s'|s)V(s')$$
    - expressing it in matrixes:  
        $$\begin{bmatrix}
            V(s_1)  \\
            V(s_2)  \\
            \vdots \\
            V(s_N) 
        \end{bmatrix} = 
        \begin{bmatrix}
            R(s_1)  \\
            R(s_2)  \\
            \vdots \\
            R(s_N) 
        \end{bmatrix} +
        \begin{bmatrix}
            p(s_1|s_1) & p(s_2|s_1) & p(s_3|s_1) & \dots  & p(s_N|s_1) \\
            p(s_1|s_2) & p(s_2|s_2) & p(s_3|s_2) & \dots  & p(s_N|s_2) \\
            \vdots & \vdots & \vdots & \ddots & \vdots \\
            p(s_1|s_N) & p(s_2|s_N) & p(s_3|s_N) & \dots  & p(s_N|s_N)
        \end{bmatrix}
        \begin{bmatrix}
            V(s_1)  \\
            V(s_2)  \\
            \vdots \\
            V(s_N) 
        \end{bmatrix}$$

        $$ \Rightarrow V = R + \gamma PV $$

        $$ \Rightarrow V - \gamma PV = R $$

        $$ \Rightarrow (1 - \gamma P)V = R $$

        $$ \Rightarrow V = (I - \gamma P)^{-1}R $$

    - pros and cons: solving $V$ directly but requires $O(N^3)$    

- by **iterative(dynamic programming) way**:
    - for all $s \in S$
    - initiate $V_0(s) = 0$ 
    - start with k = 1 until converge: $$ V_k(s) = R(s) + \gamma \sum_{s' \in S} p(s'|s)V_{k-1}(s') $$ 
    
        - e.g. $V_k(s_1) = R(s_1) + \gamma \begin{bmatrix}
            p(s_1|s_1) & p(s_2|s_1) & p(s_3|s_1) & \dots  & p(s_N|s_1)
        \end{bmatrix} \begin{bmatrix}
            V_{k-1}(s_1)  \\
            V_{k-1}(s_2)  \\
            \vdots \\
            V_{k-1}(s_N) 
        \end{bmatrix}$
    - pros and cons: $O(N^2)$ for each iteration

### 3. Markov Decision Process
**Markov Decision Process** is **Markov Reward Process plus actions**.

#### Definition
--------
|MDP|
|:--|
|$S$ is a set of states $s_t \in S$|
|$A$ is a set of actions $a \in A $|
|$P$ is dynamics/transition model that specifies $p(s'|{s,a})$ for **each state and each action** |
|$R$ is a reward function that specifies current reward $R(s,a)$ for **each state and each action** |
|$\gamma$ is the discount factor that $\in [0,1]$|

For each state, $P$ and $R$ are defined differently for each action, e.g. for $s_1,a_1$, 
$$ P(s_1,a_1) = \begin{bmatrix} p(s_1|s_1,a_1) & p(s_2|s_1,a_1) & \dots & p(s_N|s_1,a_1) \end{bmatrix} \\ 
            R(s_1,a_1)$$

### 4. Markov Desicion Process + policy
We often navigate the actions in **MDP with policy**, otherwise the agent would just take random actions. Policy can be deterministic or stochastic, but in general it's expressed as a conditional distribution, i.e. $\pi(a|s) = p(a|s)$.
#### Policy
---------
- **deterministic policy**: 

    For example, for a policy that says "from state 1, must take action 1": $$\pi(a_1|s_1) = 1. $$    
    If taking action 1 ends up in state 2, then the states dynamics for $s_1$ is    
    $$ P(s_1,a_1) = \begin{bmatrix}
                p(s_1|s_1,a_1) & p(s_2|s_1,a_1) & \dots & p(s_N|s_1,a_1)
            \end{bmatrix} = 
            \begin{bmatrix}
                0 & 1 & \dots & 0
            \end{bmatrix} $$ 
     $$ P(s_1,a_2) = \begin{bmatrix}
                p(s_1|s_1,a_2) & p(s_2|s_1,a_2) & \dots & p(s_N|s_1,a_2)
            \end{bmatrix} = 
            \begin{bmatrix}
                0 & 0 & \dots & 0
            \end{bmatrix} $$ $$ \dots $$
            
- **stochastic policy**: 

    For example, for a policy that says "from state 1, take action 1 or action 2 with same probabilities": $$\pi(a_1|s_1) = \pi(a_2|s_1) = 0.5. $$    
    If taking action 1 ends up in state 2, and action 2 stays in state 1, then the states dynamics for $s_1$ is    
    $$ P(s_1,a_1) = \begin{bmatrix}
                p(s_1|s_1,a_1) & p(s_2|s_1,a_1) & \dots & p(s_N|s_1,a_1)
            \end{bmatrix} = 
            \begin{bmatrix}
                0 & 1 & \dots & 0
            \end{bmatrix} $$
    $$ P(s_1,a_2) = \begin{bmatrix}
                p(s_1|s_1,a_1) & p(s_2|s_1,a_1) & \dots & p(s_N|s_1,a_1)
            \end{bmatrix} = 
            \begin{bmatrix}
                1 & 0 & \dots & 0
            \end{bmatrix} $$ $$ \dots $$


A sample deterministic policy looks like: $\overbrace{a_1, a_1, a_2, a_3, ...}^\text{|S|}$

Let $|A|$ denote the number of actions, $|S|$ denote the number of states, then there are $|A|^{|S|}$ deterministic policies in total. (How to get that number: in each state, there are $|A|$ we can take, so there are $\overbrace{|A|*|A|*...}^\text{|S|} = |A|^{|S|}$ deterministic policies)



#### Definition
---------
Again let's define MDP with policy more formally: 

|MDP with policy|
|:-------------|
|$S$ is a set of states $s_t \in S$|
|$A$ is a set of actions $a \in A $|
|$P^{\pi}$ is dynamics/transition model that specifies $p^{\pi}(s'|{s})$ **under a certain policy**|    
|$R^{\pi}$ is a reward function that specifies current reward $R^{\pi}(s)$ **under a certain policy**|
|$\gamma$ is the discount factor that $\in [0,1]$|

$p^{\pi}(s'|{s})$ and $R^{\pi}(s)$ are not defined directly in the environment, but we can construct it as follow:
$$ p^{\pi}(s'|s) = \sum_{a \in A} \pi(a|s)p(s'|s,a) \\
R^{\pi}(s) = \sum_{a \in A} \pi(a|s)R(s,a) $$
For example, using the deternimistic example from above, 
$$p^{\pi}(s'|s_1) = \pi(a_1|s_1) P(s_1,a_1) + \pi(a_2|s_1) P(s_1,a_2) + ... \\
= 1 * \begin{bmatrix} 0 & 1 & \dots & 0 \end{bmatrix} + 0 * \begin{bmatrix} 1 & 0 & \dots & 0 \end{bmatrix} \\
= \begin{bmatrix} 0 & 1 & \dots & 0 \end{bmatrix} $$

This is actually the same as $p(s'|s,a_{\pi})$, where $a_{\pi}$ is action specified by $\pi$ for state $s$.


$$R^{\pi}(s_1) = \pi(a_1|s_1)R(s_1,a_1) + \pi(a_2|s_1)R(s_1,a_2) + ... \\
= R(s_1,a_1) $$      

This is actually the same as $R(s,a_{\pi})$, where $a_{\pi}$ is action specified by $\pi$ for state $s$.


#### Value function
---------
If we compare this with the definition of MRP, you will find it exactly the same as MRP. **$\Rightarrow$ MDP + policy = MRP**. **So we can compute value exactly the way same as MRP.** In practise, we often use the **iterative** way:
- for all $s \in S$
- initiate $V_0^{\pi}(s) = 0$ 
- start with k = 1 until converge: $$ V_k^{\pi}(s) = R^{\pi}(s) + \gamma \sum_{s' \in S} p^{\pi}(s'|s)V_{k-1}^{\pi}(s') $$ 
    
using the same deternimistic example from above: 
    $$V_k^{\pi}(s_1) = R^{\pi}(s_1) + \gamma \begin{bmatrix} p^{\pi}(s_1|s_1) & p^{\pi}(s_2|s_1) & p^{\pi}(s_3|s_1) & \dots  & p^{\pi}(s_N|s_1) \end{bmatrix} 
            \begin{bmatrix}
                V_{k-1}^{\pi}(s_1)  \\
                V_{k-1}^{\pi}(s_2)  \\
                \vdots \\
                V_{k-1}^{\pi}(s_N) 
            \end{bmatrix} \\
                    = R^{\pi}(s_1) + \gamma \begin{bmatrix} 0 & 1 & \dots & 0 \end{bmatrix} \begin{bmatrix}
                V_{k-1}^{\pi}(s_1)  \\
                V_{k-1}^{\pi}(s_2)  \\
                \vdots \\
                V_{k-1}^{\pi}(s_N) 
            \end{bmatrix}$$

### Policy Search 
-----------
Finally, we can find the optimal policy that miaximises the value function, i.e. $$\pi^*(s) = argmax_{\pi}V^{\pi}(s) $$

There exists a unique optimal value function (but not neccassarily unique optimal policy). Optimal policy for a MDP in infinite time horizon is deterministic and stationary. We can do an exhaustive search on policy and compare the value or we can improve an initial policy until its value function converges. There are 2 ways of doing the later method: **policy iteration** (pi) and **value iteration** (vi).

Let's first introduce a new function, **state-action value (Q value)**: 

$$Q^{\pi}(s,a) = R(s,a) + \gamma \sum_{s' \in S}P(s'|s,a)V^{\pi}(s'), \forall a \in A$$

It's just slightly different to the **state value** that we've seen earlier, $V^{\pi}(s) = R^{\pi}(s) + \gamma \sum_{s' \in S}P^{\pi}(s'|s)V^{\pi}(s')$ 

#### 1. Policy Iteration
Policy iteration involves 3 steps:
- policy valuation, which is to compute $V^{\pi}(s)$
- policy improvement, where we take actions that maximise the Q value for each state, i.e. $$ \pi_{i+1}(s) = argmax_a Q^{\pi_i}(s,a)$$
    - this will allow monotonically improve the policy since $Q^{\pi_i}(s,a) \geqslant Q^{\pi_i}(s,\pi_i(a))$
- policy iteration: 
    - for all $s \in S$, initialise $\pi_0(s)$
    - start with $i=0$ until $||\pi_i - \pi_{i+1}||_{l1} = 0$ (no changes in policy)
        - policy valuation $V^{\pi_i}(s)$
        - policy imporvement $\pi_{i+1}$
        - i = i + 1

#### 2. Value Iteration
- for all $s \in S$, initialise $V_0(s)$ with zeros 
- start with $k=1$ until $V(s)$ converges
     - for each $s \in S$: $V_{k+1}(s) = max_aQ(s,a)$
     - k = k + 1
- policy extraction: $\pi_{k+1}(s) = argmax_aQ(s,a)$




### Other MDPs
-------------
Depending on how agent state $s_t$ is defined, we have different types of MDP:
- Full Observability / Markov Decision Process (MDP):
    - Agent state uses observation from world state: $s_t = o_t$
    - that is, we assume current observation is sufficient statistic of history, i.e. $o_t = h_t$ 
- Partial Observability / Partially Observable Markov Decision Process (POMDP):
    - Agent state is not the same as the world state
    - Agent contructs its own state: use $s_t = h_t$, or RNN, or beliefs of world state ...
    - when?: when we don't have access to all information like in poker game
- Bandits:
    - actions have no influence on next observations (so actions don't affect real world) 
    - so actions are independent of one another 
    - there are no delayed rewards
    - as opposed to bandits, MDP and POMDP actions influence next observation (so credict assignment and strategic actions might required)
- Continuous-state MDP:
    - requires optimal control
    
  

## Policy-Based
** To be completed

## Value-Based
** To be completed